#### **Под катом два продюсера - TOPIC 1 (CONTENT) и TOPIC 2 (FEEDBACK)** - вынесено в два ноутбука - для тестирования

In [1]:
import $ivy.`org.apache.kafka::kafka-streams-scala:2.4.0`
import $ivy.`org.slf4j:slf4j-nop:1.7.21` 
import $ivy.`com.lihaoyi::upickle:0.7.1`

import $ivy.$                                            

import $ivy.$                            

import $ivy.$                           

<br>`чтение из json-файла и запись в топик  - симуляция стримингового источника через наполнение кафки`

In [2]:
def writeToKafka(FILE: String, TOPIC: String, waitEveryN: Int = 10000, waitSecond: Int = 5) = {
    
    import scala.io.Source
    import java.util.Properties
    import org.apache.kafka.clients.producer._
    
    val kafkaProps = Map(
        "bootstrap.servers" -> "172.27.1.16:9092",
        "key.serializer"    -> "org.apache.kafka.common.serialization.StringSerializer",
        "value.serializer"  -> "org.apache.kafka.common.serialization.StringSerializer"
    )
    val props = new Properties()
    kafkaProps.foreach {case (k,v) => props.put(k,v)}    
    
    val producer = new KafkaProducer[String, String](props)
    var counter   = 0
//     var waitEveryN = 10000
//     var waitSecond = 5
    for (line <- Source.fromFile(FILE).getLines) {
        val timestamp = ujson.read(line).obj("timestamp").num.toLong
        val key       = ujson.read(line).obj("key").str
        val value     = ujson.read(line).obj("value").toString
            
        if (timestamp > 0) {
            val record = new ProducerRecord(TOPIC, null, timestamp, key, value)
            producer.send(record)
            counter += 1
            if (counter%waitEveryN == 0) {
                Thread.sleep(waitSecond*1000)
            }
        } 
          
    } 
    println(s"was sent $counter messages to topic: $TOPIC")
    producer.close()    
}

defined function writeToKafka

<br>**Тестовые топики** (в .json файлах Topik_1 ~1k Topik_2 ~2k записей)

In [8]:
// TOPIC 1 - CONTENT PRODUCER

writeToKafka(
    FILE="../data/processed/stg_content_test/content_test.json", 
    TOPIC="content_topic_test", 
    waitEveryN=1000, 
    waitSecond=2
)

was sent 1000 messages to topic: content_topic_test


In [ ]:
// TOPIC 2 - FEEDBACK PRODUCER

// writeToKafka(
//     FILE="../data/processed/stg_feedback_test/feedback_test.json", 
//     TOPIC="feedback_topic_test", 
//     waitEveryN=3000, 
//     waitSecond=2
// )

<br>**ПРОДЮСЕР-1** (~3kk записей)

In [11]:
writeToKafka(
    FILE="../data/processed/stg_content/content.json", 
    TOPIC="content_topic",
    waitEveryN=10000, 
    waitSecond=10
)

: 

**ПРОДЮСЕР-2** (18kk записей)

In [ ]:
// writeToKafka(
//     FILE="../data/processed/stg_feedback/feedback.json", 
//     TOPIC="feedback_topic",
//     waitEveryN=10000, 
//     waitSecond=1
// )

`немного json-а`
<br>
https://www.lihaoyi.com/post/HowtoworkwithJSONinScala.html

In [2]:
import scala.io.Source
val filename = "../data/processed/stg_content_example/example.json"
for (line <- Source.fromFile(filename).getLines.take(5)) {
    println(line)
}

{"timestamp":1514765113000,"key":"16876204","value":{"objectType":"Post","ownerId":31419,"ownerType":"GROUP_OPEN_OFFICIAL","authorId":680017,"applicationId":0,"numCompanions":0,"numPhotos":1,"numPolls":0,"numSymbols":0,"numTokens":0,"numVideos":0,"platform":"OTHER","totalVideoLength":0,"options":["HAS_PHOTOS","IS_PART_OF_TOPIC"]}}
{"timestamp":1514765801000,"key":"289803","value":{"objectType":"Video","ownerId":14174,"ownerType":"GROUP_OPEN","authorId":905556,"applicationId":0,"numCompanions":0,"numPhotos":0,"numPolls":0,"numSymbols":32,"numTokens":2,"numVideos":0,"platform":"ANDROID","totalVideoLength":60024,"options":["HAS_TEXT","HAS_VIDEOS","IS_EXTERNAL_SHARE"]}}
{"timestamp":1514766849000,"key":"18216307","value":{"objectType":"Post","ownerId":7221,"ownerType":"GROUP_OPEN","authorId":678857,"applicationId":0,"numCompanions":0,"numPhotos":1,"numPolls":0,"numSymbols":43,"numTokens":5,"numVideos":0,"platform":"WEB","totalVideoLength":0,"options":["HAS_TEXT","HAS_PHOTOS","IS_PART_OF_TO

import scala.io.Source

filename: String = "../data/processed/stg_content_example/example.json"

In [3]:
import scala.io.Source
val filename = "../data/processed/stg_content/content.json"
for (line <- Source.fromFile(filename).getLines.take(5)) {
    println(line)
}

{"timestamp":1514765113000,"key":"16876204","value":{"objectType":"Post","ownerId":31419,"ownerType":"GROUP_OPEN_OFFICIAL","authorId":680017,"applicationId":0,"numCompanions":0,"numPhotos":1,"numPolls":0,"numSymbols":0,"numTokens":0,"numVideos":0,"platform":"OTHER","totalVideoLength":0,"options":["HAS_PHOTOS","IS_PART_OF_TOPIC"]}}
{"timestamp":1514765801000,"key":"289803","value":{"objectType":"Video","ownerId":14174,"ownerType":"GROUP_OPEN","authorId":905556,"applicationId":0,"numCompanions":0,"numPhotos":0,"numPolls":0,"numSymbols":32,"numTokens":2,"numVideos":0,"platform":"ANDROID","totalVideoLength":60024,"options":["HAS_TEXT","HAS_VIDEOS","IS_EXTERNAL_SHARE"]}}
{"timestamp":1514766849000,"key":"18216307","value":{"objectType":"Post","ownerId":7221,"ownerType":"GROUP_OPEN","authorId":678857,"applicationId":0,"numCompanions":0,"numPhotos":1,"numPolls":0,"numSymbols":43,"numTokens":5,"numVideos":0,"platform":"WEB","totalVideoLength":0,"options":["HAS_TEXT","HAS_PHOTOS","IS_PART_OF_TO

import scala.io.Source

filename: String = "../data/processed/stg_content/content.json"

In [4]:
import scala.io.Source
val filename = "../data/processed/stg_feedback_example/example.json"
for (line <- Source.fromFile(filename).getLines.take(5)) {
    println(line)
}

{"timestamp":1517432400333,"key":"224685_30710100","value":{"membership_status":"A","feedback":["Clicked","Unliked","Liked"]}}
{"timestamp":1517432400675,"key":"8835387_36283522","value":{"membership_status":"A","feedback":["Ignored"]}}
{"timestamp":1517432400687,"key":"9505740_20360723","value":{"membership_status":"A","feedback":["Ignored"]}}
{"timestamp":1517432401051,"key":"8108256_19895436","value":{"membership_status":"A","feedback":["Ignored"]}}
{"timestamp":1517432402347,"key":"249777_24097819","value":{"membership_status":"A","feedback":["Disliked"]}}


import scala.io.Source

filename: String = "../data/processed/stg_feedback_example/example.json"

In [5]:
import scala.io.Source
val filename = "../data/processed/stg_feedback/feedback.json"
for (line <- Source.fromFile(filename).getLines.take(5)) {
    println(line)
}

{"timestamp":1517432400333,"key":"224685_30710100","value":{"membership_status":"A","feedback":["Clicked","Unliked","Liked"]}}
{"timestamp":1517432400675,"key":"8835387_36283522","value":{"membership_status":"A","feedback":["Ignored"]}}
{"timestamp":1517432400687,"key":"9505740_20360723","value":{"membership_status":"A","feedback":["Ignored"]}}
{"timestamp":1517432401051,"key":"8108256_19895436","value":{"membership_status":"A","feedback":["Ignored"]}}
{"timestamp":1517432402347,"key":"249777_24097819","value":{"membership_status":"A","feedback":["Disliked"]}}


import scala.io.Source

filename: String = "../data/processed/stg_feedback/feedback.json"

In [6]:
val json_first = Source.fromFile(filename).getLines.toList(0)
val data = ujson.read(json_first)

json_first: String = "{\"timestamp\":1517432400333,\"key\":\"224685_30710100\",\"value\":{\"membership_status\":\"A\",\"feedback\":[\"Clicked\",\"Unliked\",\"Liked\"]}}"
data: ujson.Value.Value = Obj(
  Map(
    "timestamp" -> Num(1.517432400333E12),
    "key" -> Str("224685_30710100"),
    "value" -> Obj(
      Map(
        "membership_status" -> Str("A"),
        "feedback" -> Arr(ArrayBuffer(Str("Clicked"), Str("Unliked"), Str("Liked")))
      )
    )
  )
)

In [7]:
data.obj("timestamp").num.toInt

res6: Int = 2147483647

In [8]:
data.obj("key").str

res7: String = "224685_30710100"

In [9]:
data.obj("value").toString == ujson.write(data.obj("value"))

res8: Boolean = true

In [10]:
ujson.write(data.obj("value"))

res9: String = "{\"membership_status\":\"A\",\"feedback\":[\"Clicked\",\"Unliked\",\"Liked\"]}"

In [11]:
println(ujson.write(data.obj("value").obj, indent = 4))

{
    "membership_status": "A",
    "feedback": [
        "Clicked",
        "Unliked",
        "Liked"
    ]
}


<br> **Notes**

In [ ]:
// import $ivy.`org.apache.kafka::kafka-streams-scala:2.4.0`
// import $ivy.`org.slf4j:slf4j-nop:1.7.21` 

// `Запись в тестовый kafka-топик`
// <br>
// https://gist.github.com/fancellu/f78e11b1808db2727d76

// def writeToKafkaTextTopic() = {    
//     import java.util.Properties
//     import org.apache.kafka.clients.producer._
//     val TOPIC="text_topic"
//     val kafkaProps = Map(
//         "bootstrap.servers" -> "172.27.1.16:9092",
//         "key.serializer"    -> "org.apache.kafka.common.serialization.StringSerializer",
//         "value.serializer"  -> "org.apache.kafka.common.serialization.StringSerializer"
//     )
//     val props = new Properties()
//     kafkaProps.foreach {case (k,v) => props.put(k,v)}
//     val producer = new KafkaProducer[String, String](props)
//     val SimpleDate = new java.text.SimpleDateFormat("yyyy-MM-dd")
//     for (i<- 1 to 30) {
//         val string: String = f"2020-11-${i}%02d"
//         val datetime = SimpleDate.parse(string)
//         val epoch: Long = datetime.getTime
//         if (i%2 == 0) {
//             val record = new ProducerRecord(TOPIC, null, epoch, f"key ${i.toBinaryString}", s"hello ${i} - ${string} - ${epoch.toString}")
//             producer.send(record)
//         } 
//     }
//     producer.close()
// }
// writeToKafkaTextTopic()